# SHFE Silver (AG) Advanced Market Analysis - Enhanced
## Quantitative Framework for Options Trading

**Author:** Senior Quantitative Researcher  
**Target:** SHFE AG2604 Futures & Options Chain  
**Methodology:** SABR Calibration, 3D Volatility Surface, Greeks Analysis, Dealer Gamma Exposure

---

### Analysis Framework

1. **Smart Data Ingestion**: ATM-focused option chain (±20% from spot)
2. **Futures Analysis**: Basis structure and speculative interest metrics
3. **SABR Calibration**: Rigorous volatility surface modeling
4. **3D Volatility Surface**: Interactive visualization
5. **Advanced Greeks**: Delta/Gamma/Vega profiles
6. **Market Microstructure**: Dealer positioning and gamma profiles
7. **Strategic Synthesis**: Actionable trading insights

In [ ]:
# Core Dependencies
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Tuple
from datetime import datetime, timedelta
import scipy.optimize as optimize
from scipy.stats import norm
from scipy.interpolate import CubicSpline, interp1d, griddata

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Wind API
from WindPy import w

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All dependencies loaded successfully")

## 1. Data Structures

Define typed data structures for robust data handling.

In [ ]:
@dataclass
class FuturesSnapshot:
    """SHFE Futures Contract Snapshot"""
    contract_code: str
    price: float
    bid: float
    ask: float
    volume: float
    open_interest: float
    timestamp: datetime
    
    @property
    def mid_price(self) -> float:
        return (self.bid + self.ask) / 2
    
    @property
    def spread_bps(self) -> float:
        """Bid-ask spread in basis points"""
        return 10000 * (self.ask - self.bid) / self.mid_price
    
    @property
    def volume_oi_ratio(self) -> float:
        """Speculative heat indicator"""
        return self.volume / self.open_interest if self.open_interest > 0 else 0


@dataclass
class OptionSnapshot:
    """SHFE Option Contract Snapshot"""
    contract_code: str
    option_type: str  # 'C' or 'P'
    strike: float
    price: float
    bid: float
    ask: float
    volume: float
    open_interest: float
    implied_vol: Optional[float]  # Wind's pre-calculated IV
    timestamp: datetime
    
    @property
    def mid_price(self) -> float:
        return (self.bid + self.ask) / 2 if self.bid > 0 and self.ask > 0 else self.price
    
    @property
    def is_liquid(self) -> bool:
        """Liquidity filter: Non-zero OI and reasonable bid-ask"""
        return self.open_interest > 0 and self.bid > 0 and self.ask > 0


@dataclass
class SABRParameters:
    """SABR Model Parameters"""
    alpha: float  # Initial volatility
    beta: float   # CEV exponent (0=Normal, 1=Lognormal)
    rho: float    # Correlation between asset and volatility
    nu: float     # Volatility of volatility
    f: float      # Forward price
    t: float      # Time to expiry
    
    def __post_init__(self):
        """Parameter constraints"""
        assert 0 <= self.beta <= 1, "Beta must be in [0, 1]"
        assert -1 < self.rho < 1, "Rho must be in (-1, 1)"
        assert self.alpha > 0 and self.nu > 0, "Alpha and Nu must be positive"


@dataclass
class GreeksProfile:
    """Option Greeks"""
    strike: float
    option_type: str
    delta: float
    gamma: float
    vega: float
    theta: float
    moneyness: float  # Strike / Spot


@dataclass
class GammaExposure:
    """Dealer Gamma Exposure per Strike"""
    strike: float
    call_oi: float
    put_oi: float
    call_gamma: float
    put_gamma: float
    net_gex: float  # Net Dealer GEX (negative = short gamma)
    
    @property
    def gex_notional(self) -> float:
        """GEX in notional terms (for visualization)"""
        return self.net_gex * self.strike


print("✓ Data structures defined")

## 2. Enhanced WindPy Data Ingestion

**Smart Filtering:** Only fetch options within ±20% of ATM to focus on relevant strikes.

In [ ]:
class WindDataFetcher:
    """Wind API Data Fetcher with ATM-Focused Filtering"""
    
    def __init__(self):
        self.w = w
        if not self.w.isconnected():
            self.w.start()
            print("✓ Wind API connected")
    
    def fetch_futures_snapshot(self, contract_code: str) -> FuturesSnapshot:
        """Fetch futures snapshot data"""
        fields = "rt_last,rt_bid,rt_ask,volume,oi"
        data = self.w.wsq(contract_code, fields)
        
        if data.ErrorCode != 0:
            raise ValueError(f"Wind API Error: {data.ErrorCode}")
        
        return FuturesSnapshot(
            contract_code=contract_code,
            price=data.Data[0][0],
            bid=data.Data[1][0],
            ask=data.Data[2][0],
            volume=data.Data[3][0],
            open_interest=data.Data[4][0],
            timestamp=datetime.now()
        )
    
    def fetch_option_chain_atm_focused(
        self, 
        underlying_code: str,
        spot_price: float,
        atm_range_pct: float = 0.20,
        min_strikes_per_side: int = 10
    ) -> List[OptionSnapshot]:
        """
        Fetch option chain focused on ATM strikes.
        
        Args:
            underlying_code: Futures contract code
            spot_price: Current futures price
            atm_range_pct: Percentage range around ATM (default 20%)
            min_strikes_per_side: Minimum number of strikes each side of ATM
        """
        print(f"\n🔍 Fetching option chain for {underlying_code}...")
        print(f"   ATM Price: {spot_price:,.0f}")
        print(f"   Target Range: ±{atm_range_pct*100:.0f}% [{spot_price*(1-atm_range_pct):,.0f} - {spot_price*(1+atm_range_pct):,.0f}]")
        
        # Get full option chain
        chain_data = self.w.wset(
            "optionchain",
            f"us_code={underlying_code};option_var=全部;call_put=全部"
        )
        
        if chain_data.ErrorCode != 0:
            raise ValueError(f"Option chain fetch failed: {chain_data.ErrorCode}")
        
        option_codes = chain_data.Data[0]
        strikes = chain_data.Data[1]
        call_puts = chain_data.Data[2]
        
        # Smart filtering: Get strikes around ATM
        unique_strikes = sorted(set(strikes))
        atm_idx = np.argmin([abs(s - spot_price) for s in unique_strikes])
        atm_strike = unique_strikes[atm_idx]
        
        # Calculate bounds
        lower_bound = spot_price * (1 - atm_range_pct)
        upper_bound = spot_price * (1 + atm_range_pct)
        
        # Filter strikes in range
        filtered_indices = []
        for i, strike in enumerate(strikes):
            if lower_bound <= strike <= upper_bound:
                filtered_indices.append(i)
        
        # Ensure minimum coverage
        strikes_below_atm = [s for s in unique_strikes if s < atm_strike]
        strikes_above_atm = [s for s in unique_strikes if s > atm_strike]
        
        if len(strikes_below_atm) < min_strikes_per_side:
            strikes_below_atm = unique_strikes[:min_strikes_per_side]
        else:
            strikes_below_atm = strikes_below_atm[-min_strikes_per_side:]
            
        if len(strikes_above_atm) < min_strikes_per_side:
            strikes_above_atm = unique_strikes[-min_strikes_per_side:]
        else:
            strikes_above_atm = strikes_above_atm[:min_strikes_per_side]
        
        target_strikes = set(strikes_below_atm + [atm_strike] + strikes_above_atm)
        
        # Re-filter with guaranteed coverage
        filtered_indices = [i for i, s in enumerate(strikes) if s in target_strikes]
        
        option_codes = [option_codes[i] for i in filtered_indices]
        strikes = [strikes[i] for i in filtered_indices]
        call_puts = [call_puts[i] for i in filtered_indices]
        
        print(f"   ✓ Filtered to {len(option_codes)} contracts ({len(target_strikes)} unique strikes)")
        print(f"   Strike Range: {min(target_strikes):,.0f} - {max(target_strikes):,.0f}")
        
        # Fetch market data
        if len(option_codes) == 0:
            print("   ⚠️  WARNING: No options found in range!")
            return []
        
        fields = "rt_last,rt_bid,rt_ask,volume,oi,impliedvolatility"
        market_data = self.w.wsq(option_codes, fields)
        
        options = []
        for i, code in enumerate(option_codes):
            try:
                option = OptionSnapshot(
                    contract_code=code,
                    option_type='C' if call_puts[i] == '认购' else 'P',
                    strike=strikes[i],
                    price=market_data.Data[0][i] if market_data.Data[0][i] else 0,
                    bid=market_data.Data[1][i] if market_data.Data[1][i] else 0,
                    ask=market_data.Data[2][i] if market_data.Data[2][i] else 0,
                    volume=market_data.Data[3][i] if market_data.Data[3][i] else 0,
                    open_interest=market_data.Data[4][i] if market_data.Data[4][i] else 0,
                    implied_vol=market_data.Data[5][i] if market_data.Data[5][i] else None,
                    timestamp=datetime.now()
                )
                options.append(option)
            except Exception as e:
                print(f"   Warning: Failed to parse {code}: {e}")
                continue
        
        return options
    
    def close(self):
        """Close Wind connection"""
        self.w.close()


# Initialize fetcher
fetcher = WindDataFetcher()
print("✓ Wind data fetcher initialized")

In [ ]:
# Fetch data for AG2604
UNDERLYING = "AG2604.SHF"
ATM_RANGE_PCT = 0.15  # ±15% around ATM
MIN_STRIKES = 15      # At least 15 strikes per side

try:
    # Step 1: Fetch futures snapshot
    print(f"\n{'='*70}")
    print(f"FETCHING MARKET DATA: {UNDERLYING}")
    print(f"{'='*70}")
    
    futures = fetcher.fetch_futures_snapshot(UNDERLYING)
    
    print(f"\n📊 FUTURES SNAPSHOT")
    print(f"   Price:          {futures.price:>12,.2f}")
    print(f"   Bid/Ask:        {futures.bid:>12,.2f} / {futures.ask:,.2f}")
    print(f"   Spread:         {futures.spread_bps:>12.1f} bps")
    print(f"   Volume:         {futures.volume:>12,.0f}")
    print(f"   Open Interest:  {futures.open_interest:>12,.0f}")
    print(f"   Vol/OI Ratio:   {futures.volume_oi_ratio:>12.1%}")
    
    # Step 2: Fetch ATM-focused option chain
    options = fetcher.fetch_option_chain_atm_focused(
        underlying_code=UNDERLYING,
        spot_price=futures.price,
        atm_range_pct=ATM_RANGE_PCT,
        min_strikes_per_side=MIN_STRIKES
    )
    
    # Filter liquid options
    liquid_options = [opt for opt in options if opt.is_liquid]
    
    print(f"\n📈 OPTION CHAIN SUMMARY")
    print(f"   Total Contracts:  {len(options)}")
    print(f"   Liquid Contracts: {len(liquid_options)}")
    
    # Convert to DataFrame
    options_df = pd.DataFrame([
        {
            'contract': opt.contract_code,
            'strike': opt.strike,
            'type': opt.option_type,
            'price': opt.mid_price,
            'bid': opt.bid,
            'ask': opt.ask,
            'oi': opt.open_interest,
            'volume': opt.volume,
            'iv': opt.implied_vol,
            'moneyness': opt.strike / futures.price
        }
        for opt in liquid_options
    ])
    
    unique_strikes = sorted(options_df['strike'].unique())
    atm_strike = unique_strikes[np.argmin([abs(s - futures.price) for s in unique_strikes])]
    
    print(f"\n   Strike Statistics:")
    print(f"   └─ Range:         {options_df['strike'].min():,.0f} - {options_df['strike'].max():,.0f}")
    print(f"   └─ ATM Strike:    {atm_strike:,.0f}")
    print(f"   └─ Num Strikes:   {len(unique_strikes)}")
    
    # Display sample
    print(f"\n   Sample Contracts (sorted by moneyness):")
    sample = options_df.sort_values('moneyness')[['strike', 'type', 'iv', 'oi', 'moneyness']].head(10)
    print(sample.to_string(index=False))
    
except Exception as e:
    print(f"\n❌ ERROR: Data fetch failed - {e}")
    print("\nNote: This notebook requires live Wind API access.")
    print("Generating synthetic data for demonstration...\n")
    
    # Fallback: Synthetic data with realistic AG pricing
    futures = FuturesSnapshot(
        contract_code=UNDERLYING,
        price=19000.0,  # Realistic AG price
        bid=18995.0,
        ask=19005.0,
        volume=45000,
        open_interest=180000,
        timestamp=datetime.now()
    )
    
    # Generate ATM-focused strikes
    np.random.seed(42)
    atm_strike = 19000
    strike_interval = 200
    strikes = np.arange(
        atm_strike - 15 * strike_interval,
        atm_strike + 16 * strike_interval,
        strike_interval
    )
    
    options = []
    for strike in strikes:
        moneyness = strike / futures.price
        # Realistic skew
        base_iv = 0.22
        if moneyness < 1:  # OTM puts
            iv = base_iv + 0.08 * (1 - moneyness) + np.random.normal(0, 0.005)
        else:  # OTM calls
            iv = base_iv - 0.03 * (moneyness - 1) + np.random.normal(0, 0.005)
        iv = max(0.12, min(0.40, iv))
        
        for opt_type in ['C', 'P']:
            # More OI near ATM
            distance_from_atm = abs(moneyness - 1)
            oi_base = 5000 if distance_from_atm < 0.05 else 2000 if distance_from_atm < 0.10 else 500
            oi = max(100, oi_base + np.random.normal(0, oi_base * 0.3))
            
            options.append(OptionSnapshot(
                contract_code=f"{UNDERLYING}_{opt_type}_{strike}",
                option_type=opt_type,
                strike=strike,
                price=max(10, np.random.lognormal(5, 1)),
                bid=100,
                ask=120,
                volume=max(10, np.random.lognormal(4, 1.5)),
                open_interest=oi,
                implied_vol=iv,
                timestamp=datetime.now()
            ))
    
    liquid_options = options
    options_df = pd.DataFrame([
        {
            'contract': opt.contract_code,
            'strike': opt.strike,
            'type': opt.option_type,
            'price': opt.mid_price,
            'bid': opt.bid,
            'ask': opt.ask,
            'oi': opt.open_interest,
            'volume': opt.volume,
            'iv': opt.implied_vol,
            'moneyness': opt.strike / futures.price
        }
        for opt in liquid_options
    ])
    
    print(f"✓ Synthetic data generated (Spot: {futures.price:,.0f}, {len(strikes)} strikes)")
    print(f"   Strike Range: {strikes.min():,.0f} - {strikes.max():,.0f}")

## 3. Futures Market Analysis

### 3.1 Speculative Activity Metrics

In [ ]:
# Volume/OI analysis
print(f"\n{'='*70}")
print("FUTURES MARKET MICROSTRUCTURE")
print(f"{'='*70}\n")

print(f"Spot Price:       {futures.price:>12,.2f}")
print(f"Volume/OI Ratio:  {futures.volume_oi_ratio:>12.1%}")

# Interpretation
if futures.volume_oi_ratio > 0.5:
    activity = "HIGH speculative churn (intraday momentum traders)"
elif futures.volume_oi_ratio > 0.2:
    activity = "MODERATE activity (balanced positioning)"
else:
    activity = "LOW turnover (long-term holders dominate)"

print(f"\n🔍 Interpretation: {activity}")
print(f"   Spread: {futures.spread_bps:.1f} bps → {'Liquid' if futures.spread_bps < 10 else 'Wide (caution)'}")

## 4. Advanced Options Analytics

### 4.1 Implied Volatility Structure

In [ ]:
# Separate calls and puts
calls_df = options_df[options_df['type'] == 'C'].copy()
puts_df = options_df[options_df['type'] == 'P'].copy()

# IV statistics by moneyness buckets
print(f"\n{'='*70}")
print("IMPLIED VOLATILITY STRUCTURE ANALYSIS")
print(f"{'='*70}\n")

# Define moneyness buckets
buckets = [
    (0.90, 0.95, 'Deep OTM Put'),
    (0.95, 0.98, 'OTM Put'),
    (0.98, 1.02, 'ATM'),
    (1.02, 1.05, 'OTM Call'),
    (1.05, 1.10, 'Deep OTM Call')
]

iv_analysis = []
for low, high, label in buckets:
    bucket_data = options_df[
        (options_df['moneyness'] >= low) & 
        (options_df['moneyness'] < high) &
        (options_df['iv'].notna())
    ]
    if len(bucket_data) > 0:
        avg_iv = bucket_data['iv'].mean()
        avg_oi = bucket_data['oi'].mean()
        iv_analysis.append({
            'Bucket': label,
            'Avg IV': f"{avg_iv*100:.2f}%",
            'Avg OI': f"{avg_oi:,.0f}",
            'Count': len(bucket_data)
        })

iv_analysis_df = pd.DataFrame(iv_analysis)
print(iv_analysis_df.to_string(index=False))

# Overall statistics
valid_ivs = options_df[options_df['iv'].notna()]['iv']
print(f"\n📊 Overall IV Statistics:")
print(f"   Mean:      {valid_ivs.mean()*100:.2f}%")
print(f"   Median:    {valid_ivs.median()*100:.2f}%")
print(f"   Std Dev:   {valid_ivs.std()*100:.2f}%")
print(f"   Range:     {valid_ivs.min()*100:.2f}% - {valid_ivs.max()*100:.2f}%")

### 4.2 Put-Call Parity Check

In [ ]:
# Check put-call parity: C - P = S - K (simplified, ignoring interest/dividends)
print(f"\n{'='*70}")
print("PUT-CALL PARITY ANALYSIS")
print(f"{'='*70}\n")

parity_data = []
for strike in sorted(options_df['strike'].unique()):
    call = calls_df[calls_df['strike'] == strike]
    put = puts_df[puts_df['strike'] == strike]
    
    if not call.empty and not put.empty:
        call_price = call['price'].values[0]
        put_price = put['price'].values[0]
        
        # Theoretical: C - P = S - K
        market_diff = call_price - put_price
        theoretical_diff = futures.price - strike
        deviation = market_diff - theoretical_diff
        
        parity_data.append({
            'Strike': strike,
            'C-P (Market)': market_diff,
            'S-K (Theory)': theoretical_diff,
            'Deviation': deviation,
            'Deviation %': (deviation / strike) * 100 if strike > 0 else 0
        })

parity_df = pd.DataFrame(parity_data)
if len(parity_df) > 0:
    # Show sample around ATM
    sample_parity = parity_df.iloc[
        (parity_df['Strike'] - futures.price).abs().argsort()[:5]
    ].sort_values('Strike')
    
    print("ATM Put-Call Parity Check:")
    print(sample_parity.to_string(index=False))
    
    avg_dev = parity_df['Deviation'].abs().mean()
    print(f"\n   Average Absolute Deviation: {avg_dev:.2f}")
    if avg_dev < 50:
        print("   ✓ Parity holds well (arbitrage-free)")
    else:
        print("   ⚠️  Significant deviations detected (check for arbitrage or stale quotes)")
else:
    print("   Insufficient data for parity analysis")

## 5. SABR Model Calibration

### 5.1 Mathematical Foundation

The SABR (Stochastic Alpha Beta Rho) model:

$$
\begin{align}
dF_t &= \sigma_t F_t^\beta dW_t^F \\
d\sigma_t &= \nu \sigma_t dW_t^\sigma \\
dW_t^F dW_t^\sigma &= \rho dt
\end{align}
$$

In [ ]:
class SABRModel:
    """SABR Volatility Model with Hagan's Formula"""
    
    @staticmethod
    def sabr_volatility(strike: float, params: SABRParameters) -> float:
        """
        Hagan et al. SABR implied volatility formula.
        """
        F, K, T = params.f, strike, params.t
        alpha, beta, rho, nu = params.alpha, params.beta, params.rho, params.nu
        
        # ATM case
        if abs(F - K) < 1e-6:
            term1 = alpha / (F ** (1 - beta))
            term2 = 1 + T * (
                ((1 - beta) ** 2 / 24) * (alpha ** 2 / (F ** (2 - 2 * beta)))
                + 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
                + (2 - 3 * rho ** 2) * nu ** 2 / 24
            )
            return term1 * term2
        
        # General case
        log_FK = np.log(F / K)
        FK_mid = (F * K) ** ((1 - beta) / 2)
        
        # z parameter
        z = (nu / alpha) * FK_mid * log_FK
        
        # x(z) with numerical stability
        if abs(z) < 1e-7:
            x_z = 1
        else:
            sqrt_term = np.sqrt(1 - 2 * rho * z + z ** 2)
            numerator = sqrt_term + z - rho
            denominator = 1 - rho
            x_z = np.log(numerator / denominator) / z
        
        # Main formula
        numerator = alpha
        denominator = FK_mid * (1 + ((1 - beta) ** 2 / 24) * log_FK ** 2
                                + ((1 - beta) ** 4 / 1920) * log_FK ** 4)
        
        vol_term = numerator / denominator * x_z
        
        # Time-dependent correction
        correction = 1 + T * (
            ((1 - beta) ** 2 / 24) * (alpha ** 2 / FK_mid ** 2)
            + 0.25 * rho * beta * nu * alpha / FK_mid
            + (2 - 3 * rho ** 2) * nu ** 2 / 24
        )
        
        return vol_term * correction
    
    @staticmethod
    def calibrate(
        strikes: np.ndarray,
        market_vols: np.ndarray,
        forward: float,
        time_to_expiry: float,
        beta: float = 0.7
    ) -> SABRParameters:
        """
        Calibrate SABR parameters to market IVs.
        """
        atm_vol = np.median(market_vols)
        x0 = np.array([atm_vol, 0.0, 0.3])  # [alpha, rho, nu]
        
        def objective(x):
            alpha, rho, nu = x
            if alpha <= 0 or nu <= 0 or abs(rho) >= 1:
                return 1e10
            
            try:
                params = SABRParameters(
                    alpha=alpha, beta=beta, rho=rho, nu=nu,
                    f=forward, t=time_to_expiry
                )
                
                model_vols = np.array([
                    SABRModel.sabr_volatility(K, params) for K in strikes
                ])
                
                return np.sum((model_vols - market_vols) ** 2)
            except:
                return 1e10
        
        bounds = [(0.001, 2.0), (-0.99, 0.99), (0.001, 2.0)]
        result = optimize.minimize(objective, x0, method='L-BFGS-B', bounds=bounds)
        
        alpha_opt, rho_opt, nu_opt = result.x
        
        return SABRParameters(
            alpha=alpha_opt, beta=beta, rho=rho_opt, nu=nu_opt,
            f=forward, t=time_to_expiry
        )

print("✓ SABR model implementation complete")

In [ ]:
# Prepare calibration data
calibration_range = (futures.price * 0.90, futures.price * 1.10)

calib_calls = calls_df[
    (calls_df['strike'] >= calibration_range[0]) &
    (calls_df['strike'] <= calibration_range[1]) &
    (calls_df['iv'].notna())
].sort_values('strike')

# Time to expiry (AG2604 expires in April 2026)
T = 90 / 365  # Approximately 3 months

# Calibrate
print(f"\n{'='*70}")
print("SABR MODEL CALIBRATION")
print(f"{'='*70}\n")
print(f"Using {len(calib_calls)} call options for calibration...\n")

sabr_params = SABRModel.calibrate(
    strikes=calib_calls['strike'].values,
    market_vols=calib_calls['iv'].values,
    forward=futures.price,
    time_to_expiry=T,
    beta=0.7
)

print("SABR Parameters:")
print(f"   α (alpha):     {sabr_params.alpha:>8.4f}  ← Initial volatility")
print(f"   β (beta):      {sabr_params.beta:>8.4f}  ← CEV exponent (fixed)")
print(f"   ρ (rho):       {sabr_params.rho:>+8.4f}  ← Asset-Vol correlation")
print(f"   ν (nu):        {sabr_params.nu:>8.4f}  ← Vol-of-vol\n")
print(f"   Forward:       {sabr_params.f:>8,.2f}")
print(f"   Time to Exp:   {sabr_params.t:>8.4f} years\n")

# Model fit quality
model_vols = [SABRModel.sabr_volatility(K, sabr_params) for K in calib_calls['strike'].values]
rmse = np.sqrt(np.mean((np.array(model_vols) - calib_calls['iv'].values) ** 2))
print(f"Calibration RMSE: {rmse:.4f} ({rmse*100:.2f}% vol points)")

### 5.2 Volatility Smile Visualization

In [ ]:
# Generate smooth SABR curve
strike_range = np.linspace(
    futures.price * 0.85,
    futures.price * 1.15,
    200
)
sabr_vols = [SABRModel.sabr_volatility(K, sabr_params) for K in strike_range]

# Create comprehensive volatility smile plot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Volatility Smile (Strike)',
        'Volatility Smile (Moneyness)',
        'Model Fit Quality',
        'Residuals Analysis'
    ),
    specs=[
        [{'type': 'scatter'}, {'type': 'scatter'}],
        [{'type': 'scatter'}, {'type': 'scatter'}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Top-left: Strike space
fig.add_trace(
    go.Scatter(
        x=strike_range,
        y=np.array(sabr_vols) * 100,
        mode='lines',
        name='SABR Model',
        line=dict(color='#FF6B6B', width=3)
    ),
    row=1, col=1
)

for opt_type, color, symbol in [('C', '#4ECDC4', 'circle'), ('P', '#95E1D3', 'diamond')]:
    subset = options_df[(options_df['type'] == opt_type) & (options_df['iv'].notna())]
    fig.add_trace(
        go.Scatter(
            x=subset['strike'],
            y=subset['iv'] * 100,
            mode='markers',
            name=f'Market {opt_type}',
            marker=dict(size=8, color=color, symbol=symbol)
        ),
        row=1, col=1
    )

fig.add_vline(x=futures.price, line_dash="dash", line_color="gray", 
              annotation_text="Spot", row=1, col=1)

# Top-right: Moneyness space
moneyness_range = strike_range / futures.price
fig.add_trace(
    go.Scatter(
        x=moneyness_range,
        y=np.array(sabr_vols) * 100,
        mode='lines',
        name='SABR (Moneyness)',
        line=dict(color='#FF6B6B', width=3),
        showlegend=False
    ),
    row=1, col=2
)

for opt_type, color in [('C', '#4ECDC4'), ('P', '#95E1D3')]:
    subset = options_df[(options_df['type'] == opt_type) & (options_df['iv'].notna())]
    fig.add_trace(
        go.Scatter(
            x=subset['moneyness'],
            y=subset['iv'] * 100,
            mode='markers',
            marker=dict(size=8, color=color),
            showlegend=False
        ),
        row=1, col=2
    )

fig.add_vline(x=1.0, line_dash="dash", line_color="gray", row=1, col=2)

# Bottom-left: Model fit
fig.add_trace(
    go.Scatter(
        x=calib_calls['iv'] * 100,
        y=np.array(model_vols) * 100,
        mode='markers',
        name='Calibration Points',
        marker=dict(size=10, color='#FFE66D', line=dict(width=1, color='#333')),
        showlegend=False
    ),
    row=2, col=1
)

min_iv = min(calib_calls['iv']) * 100
max_iv = max(calib_calls['iv']) * 100
fig.add_trace(
    go.Scatter(
        x=[min_iv, max_iv],
        y=[min_iv, max_iv],
        mode='lines',
        name='Perfect Fit',
        line=dict(color='gray', dash='dot'),
        showlegend=False
    ),
    row=2, col=1
)

# Bottom-right: Residuals
residuals = (np.array(model_vols) - calib_calls['iv'].values) * 100
fig.add_trace(
    go.Scatter(
        x=calib_calls['strike'],
        y=residuals,
        mode='markers',
        name='Residuals',
        marker=dict(size=8, color='#A8E6CF'),
        showlegend=False
    ),
    row=2, col=2
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=2)

# Update axes
fig.update_xaxes(title_text="Strike", row=1, col=1)
fig.update_xaxes(title_text="Moneyness (K/S)", row=1, col=2)
fig.update_xaxes(title_text="Market IV (%)", row=2, col=1)
fig.update_xaxes(title_text="Strike", row=2, col=2)

fig.update_yaxes(title_text="Implied Vol (%)", row=1, col=1)
fig.update_yaxes(title_text="Implied Vol (%)", row=1, col=2)
fig.update_yaxes(title_text="SABR IV (%)", row=2, col=1)
fig.update_yaxes(title_text="Error (% pts)", row=2, col=2)

fig.update_layout(
    title=f"SHFE AG2604 Volatility Analysis (ρ={sabr_params.rho:+.3f}, RMSE={rmse*100:.2f}%)",
    template='plotly_white',
    height=800,
    showlegend=True
)

fig.show()

### 5.3 3D Volatility Surface

Interactive 3D visualization of the volatility surface across strikes and time.

In [ ]:
# Generate 3D volatility surface
print("\n🎨 Generating 3D Volatility Surface...\n")

# Create meshgrid for strikes and time
strikes_3d = np.linspace(futures.price * 0.85, futures.price * 1.15, 50)
times_3d = np.linspace(0.01, 0.5, 30)  # 1 week to 6 months

strike_mesh, time_mesh = np.meshgrid(strikes_3d, times_3d)
vol_surface = np.zeros_like(strike_mesh)

# Calculate SABR vol for each (strike, time) pair
for i in range(len(times_3d)):
    for j in range(len(strikes_3d)):
        temp_params = SABRParameters(
            alpha=sabr_params.alpha,
            beta=sabr_params.beta,
            rho=sabr_params.rho,
            nu=sabr_params.nu,
            f=futures.price,
            t=times_3d[i]
        )
        vol_surface[i, j] = SABRModel.sabr_volatility(strikes_3d[j], temp_params) * 100

# Create 3D surface plot
fig_3d = go.Figure(data=[go.Surface(
    x=strike_mesh,
    y=time_mesh * 365,  # Convert to days
    z=vol_surface,
    colorscale='Viridis',
    colorbar=dict(title="IV (%)"),
    hovertemplate='Strike: %{x:,.0f}<br>Days: %{y:.0f}<br>IV: %{z:.2f}%<extra></extra>'
)])

# Add market data points
market_strikes = options_df[options_df['iv'].notna()]['strike'].values
market_times = np.full_like(market_strikes, T * 365)  # Current expiry
market_ivs = options_df[options_df['iv'].notna()]['iv'].values * 100

fig_3d.add_trace(go.Scatter3d(
    x=market_strikes,
    y=market_times,
    z=market_ivs,
    mode='markers',
    name='Market Data',
    marker=dict(size=5, color='red', symbol='diamond')
))

fig_3d.update_layout(
    title="SHFE AG2604 3D Volatility Surface (SABR Model)",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Days to Expiry",
        zaxis_title="Implied Volatility (%)",
        camera=dict(
            eye=dict(x=1.5, y=-1.5, z=1.2)
        )
    ),
    height=700
)

fig_3d.show()

print("✓ 3D Surface generated")
print(f"   Surface spans {len(strikes_3d)} strikes × {len(times_3d)} maturities")
print(f"   IV Range: {vol_surface.min():.2f}% - {vol_surface.max():.2f}%")

### 5.4 Skew Metrics (Risk Reversal & Butterfly)

**Risk Reversal (25Δ):** Measures directional bias in volatility

**Butterfly (25Δ):** Measures tail risk premium

In [ ]:
# Black-Scholes Delta for strike identification
def bs_delta(strike, spot, vol, T, r=0.02, opt_type='C'):
    d1 = (np.log(spot / strike) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    if opt_type == 'C':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Find 25-delta strikes
atm_iv = SABRModel.sabr_volatility(futures.price, sabr_params)
target_delta = 0.25

call_deltas = [(K, abs(bs_delta(K, futures.price, atm_iv, T, opt_type='C') - target_delta)) 
               for K in calls_df['strike'].values]
put_deltas = [(K, abs(bs_delta(K, futures.price, atm_iv, T, opt_type='P') + target_delta)) 
              for K in puts_df['strike'].values]

strike_25d_call = min(call_deltas, key=lambda x: x[1])[0]
strike_25d_put = min(put_deltas, key=lambda x: x[1])[0]

# Calculate metrics
iv_25d_call = SABRModel.sabr_volatility(strike_25d_call, sabr_params)
iv_25d_put = SABRModel.sabr_volatility(strike_25d_put, sabr_params)
iv_atm = atm_iv

risk_reversal = (iv_25d_call - iv_25d_put) * 100
butterfly = ((iv_25d_call + iv_25d_put) / 2 - iv_atm) * 100

print(f"\n{'='*70}")
print("VOLATILITY SKEW METRICS")
print(f"{'='*70}\n")
print(f"ATM Strike:        {futures.price:>10,.0f}")
print(f"ATM IV:            {iv_atm*100:>10.2f}%\n")
print(f"25Δ Call Strike:   {strike_25d_call:>10,.0f}")
print(f"25Δ Call IV:       {iv_25d_call*100:>10.2f}%\n")
print(f"25Δ Put Strike:    {strike_25d_put:>10,.0f}")
print(f"25Δ Put IV:        {iv_25d_put*100:>10.2f}%\n")
print(f"{'─'*70}")
print(f"Risk Reversal:     {risk_reversal:>+10.2f} vol pts")
print(f"Butterfly:         {butterfly:>+10.2f} vol pts")
print(f"{'─'*70}\n")

# Interpretation
print("🎯 INTERPRETATION:\n")
if risk_reversal < -1:
    print(f"   📉 NEGATIVE RR ({risk_reversal:.1f}) → PUT SKEW DOMINANT")
    print("      Market is pricing downside crash protection.")
elif risk_reversal > 1:
    print(f"   📈 POSITIVE RR ({risk_reversal:.1f}) → CALL SKEW DOMINANT")
    print("      Market is pricing upside breakout potential.")
else:
    print(f"   ⚖️  NEUTRAL RR ({risk_reversal:.1f}) → BALANCED SKEW")

if butterfly > 1:
    print(f"\n   🦋 POSITIVE BF ({butterfly:.1f}) → CONVEXITY DEMAND")
    print("      OTM options expensive vs ATM (tail risk premium).")
else:
    print(f"\n   🦋 LOW BF ({butterfly:.1f}) → FLAT SMILE")

print(f"\n   ρ (SABR): {sabr_params.rho:+.3f}")
if sabr_params.rho < -0.3:
    print("      → Leverage effect: Vol rises when price falls")
elif sabr_params.rho > 0.3:
    print("      → Inverse leverage: Vol rises when price rises")
else:
    print("      → Vol relatively independent of price direction")

## 6. Advanced Greeks Analysis

Calculate and visualize full Greek profiles across all strikes.

In [ ]:
class BSMGreeks:
    """Black-Scholes-Merton Greeks Calculator"""
    
    @staticmethod
    def d1(S, K, T, r, sigma):
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    @staticmethod
    def d2(S, K, T, r, sigma):
        return BSMGreeks.d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    @staticmethod
    def delta(S, K, T, r, sigma, opt_type='C'):
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        if opt_type == 'C':
            return norm.cdf(d1_val)
        else:
            return norm.cdf(d1_val) - 1
    
    @staticmethod
    def gamma(S, K, T, r, sigma):
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        return norm.pdf(d1_val) / (S * sigma * np.sqrt(T))
    
    @staticmethod
    def vega(S, K, T, r, sigma):
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        return S * norm.pdf(d1_val) * np.sqrt(T) / 100  # Per 1% change
    
    @staticmethod
    def theta(S, K, T, r, sigma, opt_type='C'):
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        d2_val = BSMGreeks.d2(S, K, T, r, sigma)
        
        term1 = -(S * norm.pdf(d1_val) * sigma) / (2 * np.sqrt(T))
        
        if opt_type == 'C':
            term2 = -r * K * np.exp(-r * T) * norm.cdf(d2_val)
        else:
            term2 = r * K * np.exp(-r * T) * norm.cdf(-d2_val)
        
        return (term1 + term2) / 365  # Per day
    
    @staticmethod
    def calculate_all(S, K, T, r, sigma, opt_type='C'):
        return GreeksProfile(
            strike=K,
            option_type=opt_type,
            delta=BSMGreeks.delta(S, K, T, r, sigma, opt_type),
            gamma=BSMGreeks.gamma(S, K, T, r, sigma),
            vega=BSMGreeks.vega(S, K, T, r, sigma),
            theta=BSMGreeks.theta(S, K, T, r, sigma, opt_type),
            moneyness=K/S
        )

# Calculate Greeks for all strikes
r = 0.02
greeks_data = []

for strike in sorted(options_df['strike'].unique()):
    sigma = SABRModel.sabr_volatility(strike, sabr_params)
    
    for opt_type in ['C', 'P']:
        greeks = BSMGreeks.calculate_all(futures.price, strike, T, r, sigma, opt_type)
        
        # Get OI for weighting
        oi_row = options_df[
            (options_df['strike'] == strike) & 
            (options_df['type'] == opt_type)
        ]
        oi = oi_row['oi'].values[0] if not oi_row.empty else 0
        
        greeks_data.append({
            'strike': strike,
            'type': opt_type,
            'delta': greeks.delta,
            'gamma': greeks.gamma,
            'vega': greeks.vega,
            'theta': greeks.theta,
            'moneyness': greeks.moneyness,
            'oi': oi,
            'delta_weighted': greeks.delta * oi,
            'gamma_weighted': greeks.gamma * oi,
            'vega_weighted': greeks.vega * oi
        })

greeks_df = pd.DataFrame(greeks_data)
print(f"\n✓ Greeks calculated for {len(greeks_df)} contracts")

### 6.1 Greeks Visualization

In [ ]:
# Create comprehensive Greeks dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Delta Profile', 'Gamma Profile', 'Vega Profile', 'Theta Profile'),
    specs=[
        [{'type': 'scatter'}, {'type': 'scatter'}],
        [{'type': 'scatter'}, {'type': 'scatter'}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Delta
for opt_type, color in [('C', '#4ECDC4'), ('P', '#FF6B6B')]:
    subset = greeks_df[greeks_df['type'] == opt_type]
    fig.add_trace(
        go.Scatter(
            x=subset['strike'],
            y=subset['delta'],
            mode='lines+markers',
            name=f'{opt_type} Delta',
            line=dict(color=color, width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )

# Gamma (same for calls and puts)
gamma_data = greeks_df[greeks_df['type'] == 'C']
fig.add_trace(
    go.Scatter(
        x=gamma_data['strike'],
        y=gamma_data['gamma'],
        mode='lines+markers',
        name='Gamma',
        line=dict(color='#95E1D3', width=2),
        marker=dict(size=6),
        showlegend=False
    ),
    row=1, col=2
)

# Vega (same for calls and puts)
fig.add_trace(
    go.Scatter(
        x=gamma_data['strike'],
        y=gamma_data['vega'],
        mode='lines+markers',
        name='Vega',
        line=dict(color='#FFE66D', width=2),
        marker=dict(size=6),
        showlegend=False
    ),
    row=2, col=1
)

# Theta
for opt_type, color in [('C', '#4ECDC4'), ('P', '#FF6B6B')]:
    subset = greeks_df[greeks_df['type'] == opt_type]
    fig.add_trace(
        go.Scatter(
            x=subset['strike'],
            y=subset['theta'],
            mode='lines+markers',
            name=f'{opt_type} Theta',
            line=dict(color=color, width=2),
            marker=dict(size=6),
            showlegend=False
        ),
        row=2, col=2
    )

# Add spot price line to all subplots
for row, col in [(1,1), (1,2), (2,1), (2,2)]:
    fig.add_vline(
        x=futures.price,
        line_dash="dash",
        line_color="gray",
        annotation_text="Spot" if row==1 and col==1 else "",
        row=row, col=col
    )

# Update axes labels
for col in [1, 2]:
    fig.update_xaxes(title_text="Strike", row=2, col=col)

fig.update_yaxes(title_text="Delta", row=1, col=1)
fig.update_yaxes(title_text="Gamma", row=1, col=2)
fig.update_yaxes(title_text="Vega", row=2, col=1)
fig.update_yaxes(title_text="Theta (per day)", row=2, col=2)

fig.update_layout(
    title="SHFE AG2604 Options Greeks Profiles",
    template='plotly_white',
    height=800,
    showlegend=True
)

fig.show()

# Print key statistics
atm_greeks = greeks_df[
    (greeks_df['strike'] == atm_strike) & 
    (greeks_df['type'] == 'C')
].iloc[0]

print(f"\n{'='*70}")
print("ATM OPTION GREEKS")
print(f"{'='*70}\n")
print(f"Strike:     {atm_strike:>10,.0f}")
print(f"Delta (C):  {atm_greeks['delta']:>10.4f}")
print(f"Gamma:      {atm_greeks['gamma']:>10.6f}")
print(f"Vega:       {atm_greeks['vega']:>10.2f}  (per 1% vol change)")
print(f"Theta:      {atm_greeks['theta']:>10.2f}  (per day)")

## 7. Dealer Gamma Exposure (GEX) Analysis

Analyze market maker positioning and volatility regime.

In [ ]:
# Calculate GEX for each strike
gex_profiles = []

for strike in sorted(options_df['strike'].unique()):
    call_row = greeks_df[(greeks_df['strike'] == strike) & (greeks_df['type'] == 'C')]
    put_row = greeks_df[(greeks_df['strike'] == strike) & (greeks_df['type'] == 'P')]
    
    if call_row.empty or put_row.empty:
        continue
    
    call_oi = call_row['oi'].values[0]
    put_oi = put_row['oi'].values[0]
    gamma = call_row['gamma'].values[0]
    
    # Dealer GEX = -Client GEX (dealers short options)
    # Multiply by 15 (kg per contract) for notional
    call_gex = -call_oi * gamma * strike * 15
    put_gex = -put_oi * gamma * strike * 15
    net_gex = call_gex + put_gex
    
    gex_profiles.append(GammaExposure(
        strike=strike,
        call_oi=call_oi,
        put_oi=put_oi,
        call_gamma=call_gex,
        put_gamma=put_gex,
        net_gex=net_gex
    ))

gex_df = pd.DataFrame([{
    'strike': g.strike,
    'call_oi': g.call_oi,
    'put_oi': g.put_oi,
    'net_gex': g.net_gex,
    'call_gex': g.call_gamma,
    'put_gex': g.put_gamma
} for g in gex_profiles])

# Identify key levels
max_negative_gex_strike = gex_df.loc[gex_df['net_gex'].idxmin(), 'strike']
max_positive_gex_strike = gex_df.loc[gex_df['net_gex'].idxmax(), 'strike']
zero_gamma_strikes = gex_df[
    gex_df['net_gex'].abs() < gex_df['net_gex'].abs().quantile(0.1)
]['strike'].values

print(f"\n{'='*70}")
print("DEALER GAMMA EXPOSURE (GEX) SUMMARY")
print(f"{'='*70}\n")
print(f"Current Spot:           {futures.price:>12,.0f}\n")
print(f"Max Negative GEX:       {max_negative_gex_strike:>12,.0f}  (Dealers SHORT Gamma)")
print(f"   Magnitude:           {gex_df['net_gex'].min():>12,.0f}")
print(f"\nMax Positive GEX:       {max_positive_gex_strike:>12,.0f}  (Dealers LONG Gamma)")
print(f"   Magnitude:           {gex_df['net_gex'].max():>12,.0f}\n")

if len(zero_gamma_strikes) > 0:
    closest_zero = zero_gamma_strikes[
        np.argmin(np.abs(zero_gamma_strikes - futures.price))
    ]
    print(f"Zero Gamma Level:       {closest_zero:>12,.0f}")
    print(f"Distance from Spot:     {abs(closest_zero - futures.price):>12,.0f}")
    print(f"                        ({abs(closest_zero - futures.price)/futures.price*100:>11.1f}%)")

### 7.1 GEX Visualization

In [ ]:
# Create GEX visualization
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Dealer Net Gamma Exposure by Strike',
        'Open Interest Distribution (Calls vs Puts)'
    ),
    row_heights=[0.6, 0.4],
    vertical_spacing=0.12
)

# Top: GEX bars
colors = ['#FF6B6B' if x < 0 else '#4ECDC4' for x in gex_df['net_gex']]
fig.add_trace(
    go.Bar(
        x=gex_df['strike'],
        y=gex_df['net_gex'] / 1e6,  # Convert to millions
        marker_color=colors,
        name='Net GEX',
        hovertemplate='Strike: %{x:,.0f}<br>Net GEX: %{y:.2f}M<extra></extra>'
    ),
    row=1, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)
fig.add_vline(
    x=futures.price,
    line_dash="solid",
    line_color="gold",
    line_width=3,
    annotation_text=f"Spot: {futures.price:,.0f}",
    annotation_position="top",
    row=1, col=1
)

# Bottom: OI distribution
fig.add_trace(
    go.Bar(
        x=gex_df['strike'],
        y=gex_df['call_oi'],
        name='Call OI',
        marker_color='#95E1D3',
        opacity=0.7
    ),
    row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=gex_df['strike'],
        y=-gex_df['put_oi'],  # Negative for visual separation
        name='Put OI',
        marker_color='#FFE66D',
        opacity=0.7
    ),
    row=2, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
fig.add_vline(x=futures.price, line_dash="dot", line_color="gray", row=2, col=1)

# Update axes
fig.update_xaxes(title_text="Strike", row=2, col=1)
fig.update_yaxes(title_text="Net GEX (Millions)", row=1, col=1)
fig.update_yaxes(title_text="Open Interest", row=2, col=1)

fig.update_layout(
    title="SHFE AG2604 Dealer Gamma Exposure Landscape",
    template='plotly_white',
    height=800,
    showlegend=True,
    hovermode='x unified'
)

fig.show()

### 7.2 Volatility Regime Classification

In [ ]:
# Determine current regime
current_gex = gex_df.iloc[
    (gex_df['strike'] - futures.price).abs().argsort()[:1]
]['net_gex'].values[0]

print(f"\n{'='*70}")
print("VOLATILITY REGIME ANALYSIS")
print(f"{'='*70}\n")

print(f"GEX at Spot: {current_gex:,.0f}\n")

if current_gex < -100000:
    regime = "⚠️  NEGATIVE GAMMA ZONE"
    regime_desc = "High Volatility Risk"
    interpretation = """   
   Dealers are SHORT GAMMA around current price.
   → Market makers must CHASE price movements
   → Amplified price swings expected
   → Trending behavior likely dominant
   → Realized vol likely EXCEEDS implied vol
   
   🎯 Trading Implications:
   • Long straddles/strangles (vol expansion plays)
   • Momentum strategies favored
   • Critical stop-loss discipline (gap risk)
   • Avoid short premium strategies
   """
elif current_gex > 100000:
    regime = "🛡️  POSITIVE GAMMA ZONE"
    regime_desc = "Volatility Suppression"
    interpretation = """   
   Dealers are LONG GAMMA around current price.
   → Market makers provide stabilizing liquidity
   → Price compression expected
   → Mean reversion forces active
   → Realized vol likely BELOW implied vol
   
   🎯 Trading Implications:
   • Short premium strategies (iron condors, butterflies)
   • Range-bound scalping opportunities
   • Fade extremes (mean reversion)
   • Tight stop-losses less critical
   """
else:
    regime = "⚖️  NEUTRAL GAMMA ZONE"
    regime_desc = "Transitional Regime"
    interpretation = """   
   Price near ZERO-GAMMA INFLECTION POINT.
   → Volatility regime UNCLEAR
   → Breakout in either direction could trigger shift
   → Monitor key technical levels closely
   
   🎯 Trading Implications:
   • Wait for regime confirmation before committing
   • Options may be mispriced if regime shift imminent
   • Directional bets carry elevated risk
   • Consider straddles on anticipated breakout
   """

print(f"{regime} ({regime_desc})")
print(interpretation)

## 8. Strategic Synthesis: Trader's Intelligence Report

---

In [ ]:
# Generate comprehensive trading note
activity_desc = (
    "HIGH speculative activity" if futures.volume_oi_ratio > 0.5 else
    "MODERATE trading activity" if futures.volume_oi_ratio > 0.2 else
    "LOW turnover (position holders)"
)

print(f"""
╔{'='*78}╗
║{' '*20}SHFE AG2604 MARKET INTELLIGENCE REPORT{' '*20}║
║{' '*30}{datetime.now().strftime('%Y-%m-%d %H:%M')}{' '*30}║
╚{'='*78}╝

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. MARKET SNAPSHOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   Futures Price:        {futures.price:>12,.2f}
   ATM Implied Vol:      {iv_atm*100:>12.2f}%
   Volume:               {futures.volume:>12,.0f}
   Open Interest:        {futures.open_interest:>12,.0f}
   Vol/OI Ratio:         {futures.volume_oi_ratio:>12.1%}
   Activity Level:       {activity_desc}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
2. VOLATILITY SURFACE DIAGNOSTICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   SABR Model Parameters:
   ├─ α (Initial Vol):        {sabr_params.alpha:.4f}
   ├─ β (CEV Exponent):       {sabr_params.beta:.4f}
   ├─ ρ (Asset-Vol Corr):     {sabr_params.rho:+.4f}
   └─ ν (Vol-of-Vol):         {sabr_params.nu:.4f}

   Model Quality:
   └─ Calibration RMSE:       {rmse*100:.2f}% vol pts

   Skew Metrics (25Δ):
   ├─ Risk Reversal:          {risk_reversal:+.2f} vol pts
   └─ Butterfly:              {butterfly:+.2f} vol pts

""")

# Skew interpretation
print("   📊 Skew Interpretation:")
if risk_reversal < -1.5:
    print(f"""   
      ⚠️  STRONG PUT SKEW (RR = {risk_reversal:.1f})
      Market is aggressively pricing downside protection.
      → OTM puts command significant premium vs calls
      → Crash hedging demand elevated
      → Bearish sentiment or tail-risk aversion
      
      ρ = {sabr_params.rho:.3f} confirms leverage effect:
      Volatility SURGES when price falls (commodity fear pattern).
   """)
elif risk_reversal > 1.5:
    print(f"""   
      🚀 STRONG CALL SKEW (RR = {risk_reversal:.1f})
      Market is pricing aggressive upside breakout potential.
      → OTM calls expensive (rally protection)
      → Bullish positioning or supply shock fears
      
      ρ = {sabr_params.rho:.3f} suggests inverse leverage:
      Volatility rises WITH price (atypical - potential squeeze signal).
   """)
else:
    print(f"""   
      ⚖️  BALANCED SKEW (RR = {risk_reversal:.1f})
      No strong directional bias in volatility premium.
      → Market relatively neutral on directional risk
      → ρ = {sabr_params.rho:.3f} (moderate correlation regime)
   """)

if butterfly > 1.5:
    print(f"""   
      🦋 ELEVATED BUTTERFLY (BF = {butterfly:.1f})
      Wings (OTM options) expensive relative to ATM.
      → Tail risk premium elevated
      → Market expects fat-tailed distribution
      → Jump/gap risk priced in
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
3. DEALER GAMMA POSITIONING
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   Current Regime:           {regime_desc}
   
   Key Strike Levels:
   ├─ Spot Price:            {futures.price:>12,.0f}
   ├─ Max Negative GEX:      {max_negative_gex_strike:>12,.0f}  (Vol Accelerator)
   ├─ Max Positive GEX:      {max_positive_gex_strike:>12,.0f}  (Gamma Wall)
   └─ Zero Gamma:            ~{closest_zero if len(zero_gamma_strikes) > 0 else 'N/A':>11}  (Regime Flip)

""")

# GEX implications
print("   🎯 Positioning Implications:")
if current_gex < -100000:
    print(f"""   
      Dealers SHORT GAMMA near spot ({futures.price:,.0f}).
      → Market makers forced to CHASE price (buy rallies, sell dips)
      → Positive feedback loop = AMPLIFIED VOLATILITY
      → Trend continuation likely once initiated
      
      ⚡ Critical Downside Level: {max_negative_gex_strike:,.0f}
      Break below triggers dealer hedge dumping → Cascade risk
      
      ⚡ Critical Upside Level: {max_positive_gex_strike:,.0f}
      Break above may shift regime to positive gamma
   """)
elif current_gex > 100000:
    print(f"""   
      Dealers LONG GAMMA near spot ({futures.price:,.0f}).
      → Market makers STABILIZE price (sell rallies, buy dips)
      → Negative feedback loop = SUPPRESSED VOLATILITY
      → Mean reversion forces dominant
      
      🛡️ Gamma Wall: {max_positive_gex_strike:,.0f}
      Price likely gravitates toward this strike (pin risk at expiry)
      Breakout difficult unless major catalyst emerges
   """)
else:
    print(f"""   
      Spot NEAR ZERO-GAMMA INFLECTION.
      → Regime shift possible with directional breakout
      → Monitor {max_negative_gex_strike:,.0f} (downside) and {max_positive_gex_strike:,.0f} (upside)
      → Volatility behavior will change post-break
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
4. ACTIONABLE TRADING STRATEGIES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

# Strategy recommendations based on regime
if risk_reversal < -2 and current_gex < -50000:
    print(f"""   
   📉 PRIMARY STRATEGY: Bearish Volatility Expansion
   
      Market Setup:
      • Strong put skew ({risk_reversal:.1f}) + Negative GEX
      • Downside acceleration risk underpriced
      • Dealer hedging will amplify moves
      
      Recommended Trade:
      • BUY {strike_25d_put:,.0f} / {strike_25d_put - 500:,.0f} Put Spread
      • Size: Conservative (gap risk present)
      • Target: Break below {max_negative_gex_strike:,.0f}
      • Stop: Above {atm_strike:,.0f} (regime shift)
      
      Profit Catalyst:
      If spot breaches {max_negative_gex_strike:,.0f}, dealer hedge selling
      creates non-linear price acceleration (positive feedback loop).
      
      Risk:
      Premium decay if market consolidates. IV already elevated.
   """)

elif risk_reversal > 1.5 and current_gex > 50000:
    print(f"""   
   📈 PRIMARY STRATEGY: Fade the Rally (Mean Reversion)
   
      Market Setup:
      • Call skew ({risk_reversal:.1f}) + Positive GEX
      • Upside capped by gamma wall at {max_positive_gex_strike:,.0f}
      • Dealer hedging will suppress moves
      
      Recommended Trade:
      • SELL {strike_25d_call:,.0f} / {strike_25d_call + 500:,.0f} Call Spread
      • Collect premium on capped upside
      • Target: Price pins toward {max_positive_gex_strike:,.0f}
      • Stop: Breakout above {max_positive_gex_strike:,.0f} + 3%
      
      Profit Scenario:
      Dealers long gamma = natural sellers into rallies.
      Price compression toward max GEX strike (pin effect).
      
      Risk:
      Major bullish catalyst could breach gamma wall.
      Losses capped by spread but painful if wrong.
   """)

else:
    print(f"""   
   ⚖️  PRIMARY STRATEGY: Non-Directional Volatility Play
   
      Market Setup:
      • Balanced skew or transitional regime
      • Uncertainty = potential mispricing
      
      Recommended Trade:
      • BUY ATM Straddle (Strike: {atm_strike:,.0f})
      • Bet on movement, not direction
      • Target: Breakout from {closest_zero if len(zero_gamma_strikes) > 0 else 'current range':,.0f}
      • Manage: Roll strikes if underlying drifts
      
      Profit Scenario:
      Regime shift (break of zero-gamma level) triggers vol expansion.
      Direction irrelevant if magnitude sufficient.
      
      Risk:
      Theta decay accelerates if range-bound.
      Need decisive move within ~2 weeks to profit.
   """)

# Secondary strategy
if butterfly > 2:
    print(f"""   
   🦋 SECONDARY STRATEGY: Sell Convexity (Butterfly Arb)
   
      Market Setup:
      • Elevated butterfly ({butterfly:.1f}) = overpriced wings
      
      Recommended Trade:
      • SELL Iron Butterfly
      • Short {atm_strike:,.0f} straddle
      • Long {strike_25d_put:,.0f} / {strike_25d_call:,.0f} strangle (wings)
      
      Rationale:
      Tail premium excessive. Collect theta, profit if price stays near ATM.
      Wings provide defined risk.
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
5. RISK MANAGEMENT FRAMEWORK
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ⚠️  Model Limitations:
   • SABR assumes continuous hedging (friction in reality)
   • GEX assumes dealers net short (verify with volume patterns)
   • Illiquid strikes may have stale IVs (check bid-ask spreads)
   • Model calibrated to current snapshot (re-calibrate daily)
   
   ⚠️  Execution Risks:
   • Futures spread: {futures.spread_bps:.1f} bps
   • Option spreads likely wider (check before trading)
   • Slippage on large orders can erode theoretical edge
   • Overnight gaps in commodity markets (size accordingly)
   
   ⚠️  Position Sizing Guidelines:
   • Negative Gamma Regime: Reduce size (volatility amplification)
   • Positive Gamma Regime: Can size normally (volatility dampened)
   • Near Zero-Gamma: Use wider stops (regime uncertainty)
   • Never risk >2% account on single options structure

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   📝 EXECUTIVE SUMMARY:
   
   Market Condition:
   • Futures: {futures.price:,.0f} | ATM IV: {iv_atm*100:.1f}%
   • Skew: {'PUT-biased' if risk_reversal < -1 else 'CALL-biased' if risk_reversal > 1 else 'Neutral'} (RR = {risk_reversal:+.1f})
   • Regime: {'NEGATIVE Gamma (Volatile)' if current_gex < -50000 else 'POSITIVE Gamma (Range-bound)' if current_gex > 50000 else 'NEUTRAL (Transitional)'}
   • ρ Correlation: {sabr_params.rho:+.3f}
   
   Primary Outlook:
   {'→ HIGH VOLATILITY RISK: Downside acceleration potential' if risk_reversal < -1.5 and current_gex < -50000 else '→ RANGE-BOUND: Mean reversion toward gamma wall' if current_gex > 50000 else '→ BREAKOUT PENDING: Direction uncertain, vol likely to expand'}
   
   Key Actionable Levels:
   • Support/Accelerator:  {max_negative_gex_strike:>8,.0f}
   • Current Spot:         {futures.price:>8,.0f}
   • Resistance/Wall:      {max_positive_gex_strike:>8,.0f}
   
   Position accordingly. Respect the Greeks. Size conservatively.
   Re-run analysis daily as market structure evolves.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

End of Intelligence Report.
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
""")

## 9. Appendix: Calibration Diagnostics

In [ ]:
# Detailed calibration diagnostics
calib_diagnostics = pd.DataFrame({
    'Strike': calib_calls['strike'].values,
    'Moneyness': calib_calls['strike'].values / futures.price,
    'Market IV (%)': calib_calls['iv'].values * 100,
    'SABR IV (%)': np.array(model_vols) * 100,
    'Error (%)': (np.array(model_vols) - calib_calls['iv'].values) * 100,
    'Abs Error (%)': np.abs(np.array(model_vols) - calib_calls['iv'].values) * 100
})

print(f"\n{'='*70}")
print("SABR CALIBRATION QUALITY DIAGNOSTICS")
print(f"{'='*70}\n")
print(calib_diagnostics.to_string(index=False))
print(f"\n{'─'*70}")
print(f"\nStatistics:")
print(f"   RMSE:                 {rmse*100:.3f}%")
print(f"   Mean Absolute Error:  {calib_diagnostics['Abs Error (%)'].mean():.3f}%")
print(f"   Max Absolute Error:   {calib_diagnostics['Abs Error (%)'].max():.3f}%")
print(f"   Mean Error:           {calib_diagnostics['Error (%)'].mean():.3f}%")
print(f"\n   Fit Quality: {'EXCELLENT' if rmse < 0.01 else 'GOOD' if rmse < 0.02 else 'ACCEPTABLE' if rmse < 0.03 else 'POOR'}")

## 10. Cleanup and Summary

In [ ]:
# Save key metrics for reference
summary_metrics = {
    'timestamp': datetime.now().isoformat(),
    'futures_price': futures.price,
    'atm_iv': iv_atm,
    'risk_reversal_25d': risk_reversal,
    'butterfly_25d': butterfly,
    'sabr_alpha': sabr_params.alpha,
    'sabr_rho': sabr_params.rho,
    'sabr_nu': sabr_params.nu,
    'sabr_rmse': rmse,
    'current_gex': current_gex,
    'max_negative_gex_strike': max_negative_gex_strike,
    'max_positive_gex_strike': max_positive_gex_strike,
    'regime': regime_desc
}

summary_df = pd.DataFrame([summary_metrics])

print(f"\n{'='*70}")
print("ANALYSIS SUMMARY")
print(f"{'='*70}\n")
print(summary_df.T.to_string(header=False))

print(f"\n\n{'='*70}")
print("✓ ANALYSIS COMPLETE")
print(f"{'='*70}\n")
print(f"   Contracts Analyzed:    {len(options_df)}")
print(f"   Strikes Covered:       {len(options_df['strike'].unique())}")
print(f"   Model Calibration:     SABR (β={sabr_params.beta})")
print(f"   Greeks Calculated:     Delta, Gamma, Vega, Theta")
print(f"   GEX Analysis:          Complete")
print(f"   3D Surface:            Generated")
print(f"\n   Next Steps:")
print(f"   1. Review strategy recommendations above")
print(f"   2. Verify market conditions before executing")
print(f"   3. Re-run analysis daily (market structure changes)")
print(f"   4. Monitor key GEX levels for regime shifts")
print(f"\n   Trade with discipline. Respect risk. Manage size.")
print(f"{'='*70}")

In [ ]:
# Close Wind connection
try:
    fetcher.close()
    print("\n✓ Wind API connection closed successfully")
except Exception as e:
    print(f"\n⚠️  Wind API cleanup: {e}")

print("\n" + "="*70)
print("Session complete. Good luck with your trades!")
print("="*70)